# Behavioral Clustering Analysis: San Francisco & San Jose Combined

**Project:** A Tale of Two Cities - Comparative Public Safety Analysis

**Purpose:** This notebook performs behavioral clustering using K-Means to identify distinct incident patterns across both cities based on temporal features, incident types, and contextual characteristics.

**Key Objectives:**
- Engineer temporal and behavioral features from incident data
- Identify 4-6 distinct behavioral incident profiles using K-Means
- Compare incident behavior patterns between San Francisco and San Jose
- Validate clustering quality using multiple metrics
- Generate actionable insights for public safety resource allocation

**Expected Deliverables:**
- 4-6 behavioral incident profiles with statistical characteristics
- 5+ professional visualizations (temporal patterns, radar charts, heatmaps)
- Cross-city behavioral comparison analysis
- Validation metrics (silhouette scores, statistical tests)
- Key insights on incident behavior types

---

## Methodology Overview

**Algorithm:** K-Means clustering with optimal K selection (4-6 expected)

**Feature Categories:**
1. **Temporal Features:** Hour, day of week, month, season, weekend/weekday flags
2. **Incident Characteristics:** High-level categories, subcategory distributions
3. **Contextual Features:** Police districts, geographic patterns
4. **Derived Metrics:** Incident frequency patterns, temporal densities

**Analysis Approach:**
- Feature engineering and scaling (StandardScaler)
- Optimal K determination (Elbow Method + Silhouette Analysis)
- Cluster profiling and interpretation
- Cross-city behavioral pattern comparison
- Statistical validation (chi-squared tests, silhouette scores)

---

In [2]:
## 1. Import Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Clustering and ML
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples, \
                            silhouette_score, \
                            calinski_harabasz_score, \
                            davies_bouldin_score

# Statistical tests
from scipy.stats import chi2_contingency

# Date/time handling
from datetime import datetime

# Plot settings
plt.rcParams["figure.figsize"] = (14, 10)
plt.rcParams["axes.grid"] = True
plt.rcParams["font.size"] = 11
sns.set_style("whitegrid")
sns.set_palette("husl")

## 2. Load and Combine Data

**Objective:** Load cleaned incident data from both cities and combine into a single dataset for comparative behavioral analysis.

**Data Sources:**
- San Francisco: `../data/processed/sf_incidents_cleaned.csv`
- San Jose: `../data/processed/sj_calls_cleaned.csv`

**Expected Outcome:** Combined dataset with ~200K+ incidents, city identifier column, and harmonized features for clustering.

In [5]:
# Load San Francisco data
print("\n Loading San Francisco data...")
df_sf = pd.read_csv(
    '../data/processed/sf_incidents_cleaned.csv',
    index_col='Incident DateTime',
    parse_dates=True
)
print(f"    SF loaded: {len(df_sf):,} incidents")
print(f"    Date range: {df_sf.index.min().date()} to {df_sf.index.max().date()}")

# Load San Jose data
print("\n Loading San Jose data...")
df_sj = pd.read_csv(
    '../data/processed/sj_calls_cleaned.csv',
    index_col='Incident DateTime',
    parse_dates=True
)
print(f"    SJ loaded: {len(df_sj):,} incidents")
print(f"    Date range: {df_sj.index.min().date()} to {df_sj.index.max().date()}")

# Add city identifier column
df_sf['City'] = 'San Francisco'
df_sj['City'] = 'San Jose'

# Combine datasets
print("\n Combining datasets...")
df_combined = pd.concat([df_sf, df_sj], axis=0)
print(f"    Combined dataset: {len(df_combined):,} total incidents")

# Display combined dataset info
print("\n" + "="*80)
print("COMBINED DATASET SUMMARY")
print("="*80)
print(f"\n Total incidents: {len(df_combined):,}")
print(f"    San Francisco: {len(df_sf):,} ({100*len(df_sf)/len(df_combined):.1f}%)")
print(f"    San Jose: {len(df_sj):,} ({100*len(df_sj)/len(df_combined):.1f}%)")
print(f"\n Combined date range: {df_combined.index.min().date()} to {df_combined.index.max().date()}")
print(f"\n Available columns: {df_combined.columns.tolist()}")


 Loading San Francisco data...
    SF loaded: 823,541 incidents
    Date range: 2018-01-01 to 2025-11-16

 Loading San Jose data...
    SJ loaded: 1,170,667 incidents
    Date range: 2018-01-01 to 2025-11-15

 Combining datasets...
    Combined dataset: 1,994,208 total incidents

COMBINED DATASET SUMMARY

 Total incidents: 1,994,208
    San Francisco: 823,541 (41.3%)
    San Jose: 1,170,667 (58.7%)

 Combined date range: 2018-01-01 to 2025-11-16

 Available columns: ['Incident_High_Level_Category', 'Resolution', 'Neighborhood', 'Police_District', 'Latitude', 'Longitude', 'Hour', 'Day', 'Month', 'Year', 'Day_of_Week', 'Day_of_Week_Name', 'Month_Name', 'Quarter', 'Is_Weekend', 'City']


In [7]:
temporal_features = ['Hour', 'Day', 'Month', 'Year', 'Day_of_Week', 'Day_of_Week_Name', 
                     'Month_Name', 'Quarter', 'Is_Weekend']
for feature in temporal_features:
    sf_status = True if feature in df_sf.columns else False
    sj_status = True if feature in df_sj.columns else False
    if not sf_status:
        print(f"SF missing temporal feature: {feature}")
    if not sj_status:
        print(f"SJ missing temporal feature: {feature}")
        

categorical_features = ['Incident_High_Level_Category', 'Police_District', 'Neighborhood']
for feature in categorical_features:
    sf_status = True if feature in df_sf.columns else False
    sj_status = True if feature in df_sj.columns else False
    if not sf_status:
        print(f"SF missing categorical feature: {feature}")
    if not sj_status:
        print(f"SJ missing categorical feature: {feature}")

print("\nSample Data - San Francisco:")
print(df_sf[['Hour', 'Day_of_Week_Name', 'Is_Weekend', 'Incident_High_Level_Category']].head(2))

print("\nSample Data - San Jose:")
print(df_sj[['Hour', 'Day_of_Week_Name', 'Is_Weekend', 'Incident_High_Level_Category']].head(2))



Sample Data - San Francisco:
                     Hour Day_of_Week_Name  Is_Weekend  \
Incident DateTime                                        
2025-08-27 00:37:00     0        Wednesday           0   
2025-07-17 15:00:00    15         Thursday           0   

                    Incident_High_Level_Category  
Incident DateTime                                 
2025-08-27 00:37:00                      Violent  
2025-07-17 15:00:00                        Fraud  

Sample Data - San Jose:
                     Hour Day_of_Week_Name  Is_Weekend  \
Incident DateTime                                        
2018-01-01 00:00:02     0           Monday           0   
2018-01-01 00:00:15     0           Monday           0   

                    Incident_High_Level_Category  
Incident DateTime                                 
2018-01-01 00:00:02                        Other  
2018-01-01 00:00:15                        Alarm  


### Feature Inspection Results

- **All temporal features present** in both datasets
  - Hour (0-23), Day, Month, Year
  - Day_of_Week (0-6), Day_of_Week_Name
  - Quarter (1-4), Is_Weekend (0/1)
- **All categorical features present** in both datasets
  - Incident_High_Level_Category
  - Police_District, Neighborhood
- **Feature structure is identical** between cities

---

## 3. Feature Selection for Behavioral Clustering

**Objective:** Select and prepare features that capture incident **behavior patterns** for K-Means clustering.

**Key Insight:** 
We're discovering behavioral profiles (e.g., "Late-Night Entertainment District Incidents"), not just temporal trends.

**Selected Clustering Features:**

1. **Temporal Behavior (4 features):**
   - `Hour` - Time-of-day activity pattern (0-23)
   - `Day_of_Week` - Weekday pattern (0=Monday, 6=Sunday)
   - `Is_Weekend` - Weekend flag (0/1)
   - `Quarter` - Seasonal pattern (1-4)

2. **Incident Behavior (1 feature):**
   - `Incident_High_Level_Category` - Crime type (will be encoded)

3. **Geographic Behavior (1 feature):**
   - `Police_District` - District-level patterns (will be encoded)

4. **City Identifier (1 feature):**
   - `City` - For cross-city comparison (SF vs SJ)

**Total Features for Clustering:** 7 base features → Will expand after encoding categorical variables

**Why These Features?**
- Capture **when** incidents happen (temporal)
- Capture **what type** (category)
- Capture **where generally** (district, not exact coordinates)
- Enable **cross-city comparison** (city identifier)

**Expected Outcome:** Feature matrix ready for K-Means with scaled numerical features and encoded categorical variables.

---

In [10]:
clustering_features = {
    'temporal': ['Hour', 'Day_of_Week', 'Is_Weekend', 'Quarter'],
    'categorical': ['Incident_High_Level_Category', 'Police_District'],
    'identifier': ['City']
}

print("\n Selected Feature Groups:")
print(f"\n    Temporal Features ({len(clustering_features['temporal'])}):")
for feat in clustering_features['temporal']:
    print(f"      • {feat}")

print(f"\n    Categorical Features ({len(clustering_features['categorical'])}):")
for feat in clustering_features['categorical']:
    print(f"      • {feat}")

print(f"\n    Identifier ({len(clustering_features['identifier'])}):")
for feat in clustering_features['identifier']:
    print(f"      • {feat}")

# Create working dataset with selected features
all_features = (clustering_features['temporal'] + 
                clustering_features['categorical'] + 
                clustering_features['identifier'])

df_clustering = df_combined[all_features].copy()

print(f"\n Clustering dataset created: {len(df_clustering):,} incidents")
print(f"   • Features: {len(all_features)} columns")
print(f"   • SF incidents: {len(df_clustering[df_clustering['City']=='San Francisco']):,}")
print(f"   • SJ incidents: {len(df_clustering[df_clustering['City']=='San Jose']):,}")

# Display feature value ranges
print("\n Feature Value Ranges:")
print(f"\n   Hour: {df_clustering['Hour'].min()}-{df_clustering['Hour'].max()}")
print(f"   Day_of_Week: {df_clustering['Day_of_Week'].min()}-{df_clustering['Day_of_Week'].max()}")
print(f"   Is_Weekend: {df_clustering['Is_Weekend'].unique()}")
print(f"   Quarter: {df_clustering['Quarter'].unique()}")
print(f"   Incident Categories: {df_clustering['Incident_High_Level_Category'].nunique()} unique")
print(f"   Police Districts: {df_clustering['Police_District'].nunique()} unique")


 Selected Feature Groups:

    Temporal Features (4):
      • Hour
      • Day_of_Week
      • Is_Weekend
      • Quarter

    Categorical Features (2):
      • Incident_High_Level_Category
      • Police_District

    Identifier (1):
      • City

 Clustering dataset created: 1,994,208 incidents
   • Features: 7 columns
   • SF incidents: 823,541
   • SJ incidents: 1,170,667

 Feature Value Ranges:

   Hour: 0-23
   Day_of_Week: 0-6
   Is_Weekend: [0 1]
   Quarter: [3 2 1 4]
   Incident Categories: 8 unique
   Police Districts: 12 unique


In [11]:
# Display sample
print("\n Sample of clustering dataset:")
print(df_clustering.head(3))


 Sample of clustering dataset:
                     Hour  Day_of_Week  Is_Weekend  Quarter  \
Incident DateTime                                             
2025-08-27 00:37:00     0            2           0        3   
2025-07-17 15:00:00    15            3           0        3   
2025-08-23 21:30:00    21            5           1        3   

                    Incident_High_Level_Category Police_District  \
Incident DateTime                                                  
2025-08-27 00:37:00                      Violent            Park   
2025-07-17 15:00:00                        Fraud            Park   
2025-08-23 21:30:00               Theft/Property        Northern   

                              City  
Incident DateTime                   
2025-08-27 00:37:00  San Francisco  
2025-07-17 15:00:00  San Francisco  
2025-08-23 21:30:00  San Francisco  


## 4. Feature Encoding and Scaling

**Objective:** Transform features into numerical format suitable for K-Means clustering.

**Why This Step is Critical:**
- K-Means requires all features to be numerical
- Features must be on similar scales (Hour: 0-23 vs Is_Weekend: 0-1)
- Categorical variables need encoding (e.g., "Theft" → numerical representation)

**Transformation Steps:**

1. **One-Hot Encoding for Categorical Features:**
   - `Incident_High_Level_Category` → Multiple binary columns
   - `Police_District` → Multiple binary columns
   - Prevents ordinal assumptions (no "Theft > Assault" relationship)

2. **Standardization for Numerical Features:**
   - Hour, Day_of_Week, Quarter → Mean=0, StdDev=1
   - Ensures equal weight in distance calculations
   - Uses StandardScaler from scikit-learn

3. **Feature Matrix Creation:**
   - Combine encoded categorical + scaled numerical
   - Final matrix ready for K-Means algorithm

**Expected Outcome:** Numerical feature matrix with ~20-30 dimensions (after encoding) ready for clustering.

---

In [12]:
# Step 1: One-Hot Encode Categorical Features
print("\n[1/3] One-Hot Encoding Categorical Features...")

categorical_cols = ['Incident_High_Level_Category', 'Police_District']
df_encoded = pd.get_dummies(
    df_clustering, 
    columns=categorical_cols,
    prefix=categorical_cols,
    drop_first=True  # Avoid multicollinearity
)

print("One Hot Encoding Completed.")

# Step 2: Separate target labels (City) from features
print("\n[2/3] Separating City Identifier...")
city_labels = df_encoded['City'].copy()
df_encoded = df_encoded.drop('City', axis=1)

print(f"   City labels stored separately")
print(f"   Feature matrix shape: {df_encoded.shape}")

# Step 3: Scale Numerical Features
print("\n[3/3] Scaling Features with StandardScaler...")

scaler = StandardScaler()
feature_matrix = scaler.fit_transform(df_encoded)

print(f"   Scaled feature matrix shape: {feature_matrix.shape}")
print(f"   Total features for clustering: {feature_matrix.shape[1]}")


[1/3] One-Hot Encoding Categorical Features...
One Hot Encoding Completed.

[2/3] Separating City Identifier...
   City labels stored separately
   Feature matrix shape: (1994208, 22)

[3/3] Scaling Features with StandardScaler...
   Scaled feature matrix shape: (1994208, 22)
   Total features for clustering: 22


In [14]:
# Store feature names for later interpretation
feature_names = df_encoded.columns.tolist()
print(f" Features: {feature_names}")

print("\n Feature Categories After Encoding:")
temporal_count = len([f for f in feature_names if f in ['Hour', 'Day_of_Week', 'Is_Weekend', 'Quarter']])
category_count = len([f for f in feature_names if 'Incident_High_Level_Category' in f])
district_count = len([f for f in feature_names if 'Police_District' in f])

print(f"   Temporal features: {temporal_count}")
print(f"   Incident category features: {category_count}")
print(f"   Police district features: {district_count}")
print(f"   Total: {len(feature_names)}")

print("\n" + "="*80)
print("FEATURE PREPARATION COMPLETE")
print("="*80)
print(f"\nReady for K-Means clustering:")
print(f"   Incidents: {feature_matrix.shape[0]:,}")
print(f"   Features: {feature_matrix.shape[1]}")
print(f"   Data type: Scaled numerical matrix")
print("="*80)

 Features: ['Hour', 'Day_of_Week', 'Is_Weekend', 'Quarter', 'Incident_High_Level_Category_Disturbance/Suspicious', 'Incident_High_Level_Category_Fraud', 'Incident_High_Level_Category_Non-Criminal/Admin', 'Incident_High_Level_Category_Other', 'Incident_High_Level_Category_Theft/Property', 'Incident_High_Level_Category_Traffic/Vehicle', 'Incident_High_Level_Category_Violent', 'Police_District_Central', 'Police_District_Ingleside', 'Police_District_Mission', 'Police_District_Northern', 'Police_District_Out of SF', 'Police_District_Park', 'Police_District_Richmond', 'Police_District_San Jose', 'Police_District_Southern', 'Police_District_Taraval', 'Police_District_Tenderloin']

 Feature Categories After Encoding:
   Temporal features: 4
   Incident category features: 7
   Police district features: 11
   Total: 22

FEATURE PREPARATION COMPLETE

Ready for K-Means clustering:
   Incidents: 1,994,208
   Features: 22
   Data type: Scaled numerical matrix


### Feature Encoding and Scaling

1. **One-hot encoded** categorical variables (Incident_High_Level_Category, Police_District)
2. **Separated** City identifier for post-clustering analysis
3. **Standardized** all features using StandardScaler (mean=0, std=1)
4. **Created** final numerical feature matrix for K-Means

## 5. Determine Optimal Number of Clusters (K)

**Objective:** Use Elbow Method and Silhouette Analysis to determine the optimal number of behavioral profiles (clusters).

**Methods Used:**

1. **Elbow Method:**
   - Plots inertia (within-cluster sum of squares) vs. K
   - Look for "elbow" where inertia decrease slows down
   - Indicates diminishing returns from adding more clusters

2. **Silhouette Analysis:**
   - Measures how similar incidents are to their own cluster vs. other clusters
   - Score range: -1 (poor) to +1 (excellent)
   - Higher average silhouette score indicates better-defined clusters

3. **Additional Validation Metrics:**
   - Calinski-Harabasz Score: Ratio of between-cluster to within-cluster variance (higher is better)
   - Davies-Bouldin Index: Average similarity between clusters (lower is better)

**Testing K Range:** 2 to 10 clusters

---

In [ ]:
k_range = range(2, 11)

inertias = []
silhouette_scores = []
calinski_scores = []
davies_bouldin_scores = []

print(f"\nTesting K values from {min(k_range)} to {max(k_range)}...")
print("This may take several minutes for large datasets...\n")

# Test each K value
for k in k_range:
    print(f"Testing K={k}...", end=" ")
    
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=300)
    cluster_labels = kmeans.fit_predict(feature_matrix)
    
    # Calculate metrics
    inertia = kmeans.inertia_
    sil_score = silhouette_score(feature_matrix, cluster_labels)
    cal_score = calinski_harabasz_score(feature_matrix, cluster_labels)
    db_score = davies_bouldin_score(feature_matrix, cluster_labels)
    
    # Store results
    inertias.append(inertia)
    silhouette_scores.append(sil_score)
    calinski_scores.append(cal_score)
    davies_bouldin_scores.append(db_score)
    
    print(f"Silhouette: {sil_score:.4f}, Inertia: {inertia:,.0f}")


Testing K values from 2 to 10...
This may take several minutes for large datasets...

Testing K=2... 